In [2]:
import pandas as pd
import os
import librosa
import os
from skimage import io
import torch
import torchvision
from torch.utils.data import Dataset
import math
import IPython.display as ipd

## Reconstructing File Paths

file_path = 'Datasets/LibriSpeech/train-clean-100.index.csv'

df = pd.read_csv(file_path)

df.columns

df.head()

df.shape


df.filepath[1]

df.filepath[1][0:38]

mine = '/home/amonajemi/amonajemi/Datasets'

df['filepath'] = df['filepath'].str[37:]

df['filepath'] = mine + df['filepath']

df['filepath'].head()

from torch.utils.data import Dataset

class LibriSpeechData(Dataset):
    def __init__(self,df,transform = None, sampling_rate = 8000, offset=1.0, duration=5.0):
        self.duration = duration
        self.df = df[df.seconds > self.duration].reset_index()
        self.audio = self.df['filepath']
        self.identities=self.df['id']
        self.transform = transform
        self.sampling_rate = sampling_rate
        if self.transform is None:
            self.transform = torch.from_numpy # Call it as an object! Now transform is defined so 
                                                # we cn iterate through the batches. 

    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        audio = self.audio[index]
        self.seconds = self.df['seconds'][index]
        self.offset = random.uniform(0,self.seconds - self.duration)
        y,  sr  = librosa.load(audio, sr=self.sampling_rate, offset=self.offset, duration=self.duration)
        # The y is the one-dimensional np.array
        #that is the amplitude of the waveform at sample t
        
        #the sr is the number of samples per second of a
        #time series.
    
        time_series = self.transform(y)
        
        return time_series, torch.tensor(self.identities[index])        

df.index # Our filtering of the dataset kept the original index. This is why you see 28437 here but the
#overall length of indexes is indeed 26504 

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display


    
def energy_plot(a, hop_length = 256, frame_length = 2048):
    energy = np.array([
        sum(abs(a.numpy()[i:i+frame_length]**2))
        for i in range(0, len(a.numpy()), hop_length)])
    return plt.plot(energy / energy.max()), a.min(),a.max()

## Energy Detection 

def energy_values(a, hop_length = 256, frame_length = 2048):
    energy = np.array([
        sum(abs(a.numpy()[i:i+frame_length]**2))
        for i in range(0, len(a.numpy()), hop_length)])
    return energy
def energy_values(a, hop_length = 256, frame_length = 2048):
    energy = np.array([
        sum(abs(a[i:i+frame_length]**2))
        for i in range(0, len(a), hop_length)])
    return energy

get_energy = energy_values





In [4]:
import random 
Voices = pd.read_csv("/home/amonajemi/amonajemi/Datasets/VOiCES/VOiCES-room-1.index.csv")

Voices.columns

Voices['filepath'][40000]

Voices['filepath'][1][22:]

file_e = '/home/amonajemi/amonajemi/Datasets/VOiCES/Volumes/'

Voices['filepath'] = Voices['filepath'].str[22:]

Voices['filepath'] = file_e + Voices['filepath']
    

Voices = Voices[~Voices.filepath.str.contains("tele")]

len(Voices)

Voices.index = range(0,len(Voices))

Voices = Voices[(Voices.filepath.str.contains("sp_3549") & Voices.filepath.str.contains("none"))|(Voices.filepath.str.contains("sp_6241") & Voices.filepath.str.contains("none")) | (Voices.filepath.str.contains("sp_1212") & Voices.filepath.str.contains("musi")) | (Voices.filepath.str.contains("sp_6241") & Voices.filepath.str.contains("musi"))]

Voices.index = range(0,len(Voices))



class VoicesDataSet(Dataset):
    def __init__(self,df,transform = None, sampling_rate = 8000, offset=1.0, duration=5.0):
        self.duration = duration
        self.df = df[df.seconds > self.duration].reset_index()
        self.audio = self.df['filepath']
        self.identities=self.df['id']
        self.transform = transform
        self.sampling_rate = sampling_rate
        if self.transform is None:
            self.transform = torch.from_numpy 
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        audio = self.audio[index]
        self.seconds = self.df['seconds'][index]
        self.offset = random.uniform(0,self.seconds - self.duration)
        y,  sr  = librosa.load(audio, sr=self.sampling_rate, offset=self.offset, duration=self.duration)
    
        time_series = self.transform(y)
        
        return time_series, torch.tensor(self.identities[index])
    




In [8]:
## Fade In / Fade Out
import statistics
def fading_in(length):
    fading_in =np.linspace(0, 1, length)
    #fading_in[int(length*.50):] = 1
    return fading_in

def fading_out(length):
    fading_out = np.linspace(0,1, length)[::-1]
    #fading_out[:int(length * .50)] = 1
    return fading_out



# VAD

def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    greatest = 0
    start = 0       # For Voices change this to the max interval again like below
    end = 0
    for interval in ranges:
        diff = interval[1] - interval[0]
        if diff > greatest:
            greatest = diff
            start = interval[0]
            end = interval[1]
    return list([start, end - 1])

def zero_runs_full(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

# Step 1 : Take an instance of the Voices Class
def Voices_data_Speech(dataset_class, _data):
    data = dataset_class(_data)
    #global a
    a,b = data[random.choice(_data.index)]
    vad = VAD(a.numpy(), sr = 8000, theshold = .30)
    zero_runs_intervals = zero_runs_full(vad)
    sorted_silences = sorted(zero_runs_intervals,  key = lambda arr :  arr[1] - arr[0], reverse = True)[:2]
    max_energy = 1000
    start = 0
    end = 0
    for interval in sorted_silences:
        energy_amount = statistics.mean(get_energy(a.numpy()[interval[0] : interval[1]]))
        if energy_amount <= max_energy:
            max_energy = energy_amount
            start = interval[0]
            end = interval[1] - 1
    #plt.plot(vad)
    #plt.plot(a.numpy())
    return list([start, end])

def one_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    interval = []
    for i in range(len(ranges) - 1):
        start_one = ranges[i][1]
        upper = i+1
        end_one = ranges[upper][0]-1
        interval.append(start_one)
        interval.append(end_one)
    ranges_ones = np.array(interval).reshape(-1,2)
    
    return ranges_ones 
    


def make_new_audio(Voices_data, Lib_data, silent_begin, silent_end, silent_interval):
    if silent_begin is not 0:
        sample_audio = Voices_data[:int(silent_begin)]
      
        
        Voices_Segment1 = Voices_data[:int(silent_begin)] 
    
    half1_silent = Voices_data[int(silent_begin) : int(statistics.mean(silent_interval))]
    fade_out_half1 = half1_silent * fading_out(len(half1_silent))
    
    Libri_audio_half1 = Lib_data[:int(math.floor(len(Lib_data) / 2))]
    Libri_audio_half1 = Libri_audio_half1
    
    Libri_audio_half2 = Lib_data[int(math.floor(len(Lib_data) / 2)):]
    Libri_audio_half2 = Libri_audio_half2
    
    Libri_full = np.concatenate((Libri_audio_half1 , Libri_audio_half2))
    
    half2_silent = Voices_data[int(statistics.mean(silent_interval)):int(silent_end)]
    fade_in_half2 = half2_silent* fading_in(len(half2_silent)) # Do not do fading here!
    
    
    sample_audio1 = Voices_data[int(silent_end):]

    Voices_Segment2 = Voices_data[int(silent_end):]
    if Voices_Segment1 is not None: # Put this not .all()
        new_audio = np.concatenate((Voices_Segment1, fade_out_half1,Libri_full, fade_in_half2, Voices_Segment2))
    else:
        new_audio = np.concatenate((fade_out_half1, Libri_full, fade_in_half2, Voices_Segment2))
        
    return new_audio

def audio_frames(Voices_data, Lib_data, silent_begin, silent_end, silent_interval):
    # create zeros and ones for frames 
    _Voices_seg1 = np.zeros(len(Voices_data[:int(silent_begin)]))
    _silent1 = np.zeros(len(Voices_data[int(silent_begin) : int(statistics.mean(silent_interval))]))
    _Libri = np.ones(len(Lib_data))
    _silent2 = np.zeros(len(Voices_data[int(statistics.mean(silent_interval)) : int(silent_end)]))
    _Voices_seg2 = np.zeros(len(Voices_data[int(silent_end):]))
    frames = np.concatenate((_Voices_seg1, _silent1, _Libri, _silent2, _Voices_seg2))
    #plt.plot(frames)
    return frames
    

def normalizer(data):
    series = Series(data)
    values = series.values
    values = values.reshape((len(values), 1))
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(values)
    normalized = scaler.transform(values)
    normalized = normalized.reshape(-1)
    return normalized
    
def truncate_audio(new_audio, synthetic_frames, truncate_size) : 
    right_prop_libri = sum(synthetic_frames[0 : truncate_size]) / len(synthetic_frames[0 : truncate_size])

    left_prop_libri = sum(synthetic_frames[-1 * truncate_size : ]) / len(synthetic_frames[-1*truncate_size:])
    
    if abs(right_prop_libri - 0.30) <= abs(left_prop_libri - 0.30) and abs(right_prop_libri - 0.30)<=0.15:
        truncated_audio = new_audio[0 : truncate_size]
        synthetic_frames_truncated_audio = synthetic_frames[0:truncate_size]
        
    else:
        truncated_audio = new_audio[-1 * truncate_size : ]
        synthetic_frames_truncated_audio = synthetic_frames[-1 * truncate_size : ]
    
    return truncated_audio , synthetic_frames_truncated_audio


def get_dist_libri(n_iter):
    ratio = []
    for i in range(n_iter):
        sample = v[random.choice(Voices.index)]
        if len(sample)==3:
            ratio.append(sum(sample[2]) / len(sample[2]))
    return ratio

def truncate_until_prop(new_audio , synthetic_frames , truncated_length, prop_min , prop_max):
    audio_length = len(new_audio)
    end_point = audio_length - truncated_length
    Flag = True
    while Flag:
        rand_index = random.randint(0,end_point)
        prop = sum(synthetic_frames[rand_index : rand_index + truncated_length]) / truncated_length
        if prop_min <=prop<=prop_max:
            Flag = False
            new_audio = new_audio[rand_index : rand_index + truncated_length]
            synthetic_regions = synthetic_frames[rand_index : rand_index + truncated_length]
    return new_audio, prop, synthetic_regions
        


In [ ]:
# Train / Test Split 

from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from random import sample
from pandas import Series
from sklearn.preprocessing import MinMaxScaler

validation_split = .2
shuffle_dataset = True
random_seed= 42
data = VoicesDataSet(Voices)
dataset_size = len(data) # len of the voices dataset 
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed) # These will perform what is assigned to validation and what is 
                        #assigned to training
    np.random.shuffle(indices)
# grab indices 
train_indices, val_indices = indices[split:], indices[:split]


# Creating PT data samplers and loaders:

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)



indices_real_train = sample(train_indices , len(train_indices) // 2)
indices_fake_train = [i for i in train_indices if i not in indices_real_train]
indices_real_val = sample(val_indices , len(val_indices) // 2)
indices_fake_val = [i for i in val_indices if i not in indices_real_val]



In [15]:
class Creating_audio(Dataset):
    def __init__(self, LibriSpeechData , VoicesDataSet, length_needed, padding,indices_real_train, 
                 indices_fake_train, indices_real_val, indices_fake_val, transform = None): #Libri, Voices)
        self.length_needed = length_needed
        self.padding = padding
        self.indices_real_train = indices_real_train
        self.indices_fake_train = indices_fake_train
        self.indices_real_val = indices_real_val
        self.indices_fake_val = indices_fake_val
        self.transform = transform
        if self.transform is None:
            self.transform = torch.from_numpy
        self.LibriSpeechData = LibriSpeechData(df) # Would need to get data here
        self.VoicesDataSet = VoicesDataSet(Voices)
        
    def __len__(self):
        return len(self.VoicesDataSet) # self.Voices.shape[0] # we cut off the Voices, may need to change this
        
    def __getitem__(self, index):
        
        if index in self.indices_real_train or index in self.indices_real_val: 
            
            a, b = self.VoicesDataSet[index]
            a = a[0:32000]
            a = self.transform(normalizer(a))
            return a , 0
            
        else:
            a , b = self.VoicesDataSet[index]
            vad = VAD(a.numpy(), sr = 8000, theshold = .30)
            zero_runs_intervals = zero_runs_full(vad)
            sorted_silences = sorted(zero_runs_intervals,  key = lambda arr :  arr[1] - arr[0], reverse = True)[:2]
                # Can make a threshold here of what the requirement of length is for a silent window (eg : 4000)
                #If there is only one silence interval, then no need for the energy loop.
            max_energy = 1000
            start = 0
            end = 0
            for interval in sorted_silences:
                energy_amount = statistics.mean(get_energy(a.numpy()[interval[0] : interval[1]]))
                if energy_amount <= max_energy:
                    max_energy = energy_amount
                    start = interval[0]
                    end = interval[1] - 1

            Voices_Silent_interval = list([start , end])

            full_Voices = a.numpy()

            data = self.LibriSpeechData
            Flag = True
            while Flag:
                y, c  = data[random.choice(data.df.index)]
                vad = VAD(y.numpy(), sr = 8000, theshold = .30)
                interval_ones = one_runs(vad)
                interval_zeros = zero_runs_full(vad) # Reverse = False for the short Libri Segments, True for the long Libri Segments                             
                sorted_intervals = sorted(interval_ones, key = lambda arr : arr[1] - arr[0], reverse = False)
                new_intervals = [i for i in sorted_intervals if 40000 not in i and 0 not in i and i[1] - i[0]>=self.length_needed]
                selected_interval=0
                intervals_passed = []
                if new_intervals:
                    for i in new_intervals:
                        start_left = interval_zeros[np.where(interval_zeros==i[0])[0]][0][0]
                        end_left = interval_zeros[np.where(interval_zeros==i[0])[0]][0][1] - 1
                        start_right = interval_zeros[np.where(interval_zeros==i[1] + 1)[0]][0][0]
                        end_right = interval_zeros[np.where(interval_zeros==i[1] + 1)[0]][0][1] - 1
                        if end_left - start_left >=self.padding and end_right - start_right >=self.padding:
                            full_interval = list([i[0] - (end_left - start_left) , i[1] + (end_right - start_right)])
                            if full_interval[1] - full_interval[0]<=22000:
                               #Putting a length requirement on LibriSpeech
                                intervals_passed.append(full_interval)
                    if intervals_passed:
                        r_interval = random.choice(intervals_passed)
                        Libri_Speech = y.numpy()[r_interval[0] : r_interval[1]]
                        Flag = False

            new_audio = make_new_audio(full_Voices, Libri_Speech, Voices_Silent_interval[0], Voices_Silent_interval[1], Voices_Silent_interval)


            synthetic_frames = audio_frames(full_Voices, Libri_Speech, Voices_Silent_interval[0], Voices_Silent_interval[1], Voices_Silent_interval)
                
            new_audio , prop, synthetic_regions = truncate_until_prop(new_audio , synthetic_frames , 32000 , 0.05 , 0.45)

            return self.transform(normalizer(new_audio)).float() , 1

In [ ]:
import warnings
import sklearn
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

In [16]:
v = Creating_audio(LibriSpeechData, VoicesDataSet, length_needed = 3000, padding = 700,
                   indices_real_train = indices_real_train , indices_fake_train = indices_fake_train, 
                   indices_real_val = indices_real_val, indices_fake_val = indices_fake_val, transform = None)

In [36]:
pp=VoicesDataSet(Voices)
pp.df.index

1
32000


In [ ]:
sample = v[random.choice(pp.df.index)]
plt.plot(sample[0].numpy())
plt.plot(sample[1])

In [ ]:
ipd.Audio(sample[0].numpy() , rate = 8000)

In [ ]:
# Adjusting collate to pack data into tuple of tensors - accepts variable length inputs 

torch.utils.data.dataloader.default_collate = (lambda default_collate = torch.utils.data.dataloader.default_collate: lambda batch: batch if all(map(torch.is_tensor, batch)) and any([tensor.size() != batch[0].size() for tensor in batch]) else default_collate(batch))()

In [38]:
torch.utils.data.dataloader.default_collate = torch.utils.data.dataloader.default_collate

In [39]:
batch_size = 24

train_loader = torch.utils.data.DataLoader(v, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers=4, drop_last = True)
validation_loader = torch.utils.data.DataLoader(v, batch_size=batch_size,
                                                sampler=valid_sampler, num_workers=4, drop_last = True)

In [ ]:
batch = next(iter(train_loader))

In [ ]:
time_series , labels = batch
print(time_series)
print(labels)

In [54]:
from torch import nn
import torch.nn.functional as F
from torch import device
from torch import optim
from torch.autograd import Variable

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 128, 80, 4)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(128, 128, 3)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(128, 256, 3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(256, 512, 3)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(4)
        #self.avgPool = nn.AvgPool1d(30) #input should be 512x30 so this outputs a 512x1
        self.fc1 = nn.Linear(512*30, 1)

    def forward(self, x):
        x = x.view(x.shape[0], 1,-1)
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
#       x = self.avgPool(x)
        x = x.view(batch_size, -1)
        #x = x.permute(0, 2, 1) #change the 512x1 to 1x512
        x = self.fc1(x)
        #x = F.softmax(x, dim=1)
        x = torch.sigmoid(x)
        return x

In [43]:
class CRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_dim = 1, batch_size = batch_size):
        super(CRNN , self).__init__()
        self.conv1 = nn.Conv1d(1, 64, 80, 4)
        self.bn1 = nn.BatchNorm1d(64)
        self.pool1= nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(64, 64, 3)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(64, 128, 3)
        self.bn3 = nn.BatchNorm1d(128)
        self.conv4 = nn.Conv1d(128, 128, 3)
        self.bn4 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(4)
        self.conv5 = nn.Conv1d(128, 256, 3)
        self.bn5 = nn.BatchNorm1d(256)
        self.conv6 = nn.Conv1d(256, 256, 3)
        self.bn6 = nn.BatchNorm1d(256)
        self.pool4 = nn.MaxPool1d(4)
        self.conv7 = nn.Conv1d(256, 512 , 3)
        self.bn7 = nn.BatchNorm1d(512)
        self.conv8 = nn.Conv1d(512, 512, 3)
        self.pool5 = nn.MaxPool1d(4)
        #shape batch size x 512 x 6
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.output_dim = output_dim
        #input size is 6 features time step
        #batch_size x 512 x 6
        self.lstm = nn.LSTM(self.input_size , self.hidden_size , self.num_layers, batch_first = True)
        self.to_linear = nn.Linear(self.hidden_size , self.output_dim)
    def init_hidden(self):
        return (Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)),
                Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)))
    
    def forward(self, x):
        x = x.view(x.shape[0], 1,-1)
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool3(x)
        x = self.conv5(x)
        x = F.relu(self.bn5(x))
        x = self.conv6(x)
        x = F.relu(self.bn6(x))
        x = self.pool4(x)
        x = self.conv7(x)
        x = F.relu(self.bn7(x))
        x = self.conv8(x)
        x = F.relu(self.pool5(x))
        #x = x.view(batch_size , -1, 1)
        lstm_out, hidden = self.lstm(x)
        logits = self.to_linear(lstm_out)
        scores = torch.sigmoid(logits)
        return scores
    

In [70]:
class CNN8(nn.Module):
    def __init__(self):
        super(CNN8, self).__init__()

        # 32000 x 1
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=3, stride=3, padding=0),
            nn.BatchNorm1d(128),
            nn.ReLU())
        # 10666 x 128
        self.conv2 = nn.Sequential(
            nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(3, stride=3))
        # 3555 x 128
        self.conv3 = nn.Sequential(
            nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(3,stride=3))
        # 1185 x 128
        self.conv4 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(3,stride=3))
        # 395 x 256
        self.conv5 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(3,stride=3))
        # 131 x 256
        self.conv6 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(3,stride=3))
        # 43 x 256
        self.conv7 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(3,stride=3))
        # 14 x 256
        self.conv8 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(3,stride=3))
        # 4 x 256
        self.fc = nn.Linear(4 * 256, 1) # 512

    def forward(self, x):
        # input x : 24 x 32000 x 1
        # expected conv1d input : minibatch_size x num_channel x width
        x = x.view(x.shape[0], 1,-1)
        # x : 24 x 1 x 32000 
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = self.conv6(out)
        out = self.conv7(out)
        out = self.conv8(out)
        #out = self.conv9(out)
        #out = self.conv10(out) 
        out = out.view(x.shape[0],-1)
        logit = self.fc(out)
        scores = torch.sigmoid(logit)

        return scores
    

In [71]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN()
model.to(device)
model_CRNN = CRNN()
mode_CRNN.to(device)
model_CNN8 = CNN8()
model_CNN8.to(device)
model_CRNN = CRNN(input_size = 6, hidden_size = 128, num_layers = 2)
model_CRNN.to(device)
optimizer_CNN8 = optim.Adam(model_CNN8.parameters(), lr = 0.001)
optimizer_CRNN = optim.Adam(model_CRNN.parameters(), lr = 0.001)
optimizer_CNN = optim.Adam(model_CNN.parameters(), lr = 0.001)
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
num_epochs = 20

In [47]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [49]:
def get_preds(preds):
    new_preds = []
    for i in preds:
        if i <=.50:
            new_preds.append(0)
        else:
            new_preds.append(1)
    return torch.tensor(new_preds)
def get_accuracy(preds , labels, batch_size):
    return preds.argmax(dim = 1).eq(labels).sum().item() / batch_size

In [72]:
#val_loss_list_CNN8 , epoch_list_CNN8, accuracy_list_CNN8 = [], [], []

In [ ]:
for epoch in range(num_epochs):
    
    running_loss_train , accuracy_train = 0 , 0
    
    model_CNN8.train()
    
    for batch_idx, batch in enumerate(train_loader):
        
        optimizer_CNN8.zero_grad()
        time_series , labels = batch
        new_time_series, new_labels = time_series.to(device), labels.to(device)
        preds = model_CNN8(new_time_series)
        train_loss = criterion(preds, new_labels.float()) # float and unsqueeze for BCE
        train_loss.backward()
        optimizer_CNN8.step()
        running_loss_train+=train_loss.item()/new_time_series.shape[0]
        #accuracy = get_accuracy(preds, new_labels, batch_size)
        accuracy = accuracy_score(labels, get_preds(preds))
        accuracy_train+=accuracy
        #recall = recall_score(new_labels.cpu(), preds.argmax(dim=1).cpu())
        recall = recall_score(labels, get_preds(preds))
        if batch_idx%50==0:
            print('Epoch : %d | Batch : %d | Train Loss : %.3f | Train Acc : %.3f | Train Recall : %.3f ' % 
                  (epoch , batch_idx, train_loss.item() / new_time_series.shape[0], accuracy, recall))
    
    print("Validation....")
    
    with torch.no_grad():
        
        model_CNN8.eval()
        
        #model_C.hidden = model_CRNN.init_hidden()
        
        running_loss_val, accuracy_val, recall_val , precision_val = 0 , 0 , 0 , 0
        
        for batch_idx_val, batch_val in enumerate(validation_loader):
                                   
            time_series_val , labels_val = batch_val
            new_time_series_val, new_labels_val = time_series_val.to(device), labels_val.to(device)
            preds_val = model_CNN8(new_time_series_val)
            val_loss = criterion(preds_val, new_labels_val.float()) # float for BCE 
            running_loss_val+=val_loss.item() / new_time_series_val.shape[0]
            #accuracy_= get_accuracy(preds_val , new_labels_val, batch_size)
            accuracy_ = accuracy_score(labels_val , get_preds(preds_val))
            accuracy_val+=accuracy_
            #precision_ = precision_score(new_labels_val.cpu() , preds_val.argmax(dim=1).cpu())
            precision_ = precision_score(labels_val , get_preds(preds_val))
            precision_val+=precision_
            #recall_ = recall_score(new_labels_val.cpu() , preds_val.argmax(dim=1).cpu())
            recall_ = recall_score(labels_val , get_preds(preds_val))
            recall_val += recall_
    
        
        
        print('Epoch : %d | Train Loss : %.3f | Train Acc : %.3f | Val Loss : %.3f | Val Acc : %.3f | Recall : %.3f | Precision : %.3f' 
              % (
                  epoch , 
                  running_loss_train / len(train_loader),
                  accuracy_train / len(train_loader),
                  running_loss_val / len(validation_loader),
                  accuracy_val / len(validation_loader),
                  recall_val / len(validation_loader),
                  precision_val / len(validation_loader)
              )
          )
                    
    epoch_list_CNN8.append(epoch)
    val_loss_list_CNN8.append(running_loss_val / len(validation_loader))
    accuracy_list_CNN8.append(accuracy_val / len(validation_loader))

/usr/local/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([24])) that is different to the input size (torch.Size([24, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch : 0 | Batch : 0 | Train Loss : 0.030 | Train Acc : 0.542 | Train Recall : 0.786 
Epoch : 0 | Batch : 50 | Train Loss : 0.027 | Train Acc : 0.625 | Train Recall : 0.462 
Epoch : 0 | Batch : 100 | Train Loss : 0.019 | Train Acc : 0.708 | Train Recall : 0.583 
Epoch : 0 | Batch : 150 | Train Loss : 0.018 | Train Acc : 0.875 | Train Recall : 0.818 
Epoch : 0 | Batch : 200 | Train Loss : 0.025 | Train Acc : 0.625 | Train Recall : 0.429 
Epoch : 0 | Batch : 250 | Train Loss : 0.015 | Train Acc : 0.958 | Train Recall : 0.900 
Epoch : 0 | Batch : 300 | Train Loss : 0.010 | Train Acc : 0.917 | Train Recall : 0.818 
Epoch : 0 | Batch : 350 | Train Loss : 0.006 | Train Acc : 1.000 | Train Recall : 1.000 
Epoch : 0 | Batch : 400 | Train Loss : 0.013 | Train Acc : 0.792 | Train Recall : 0.818 
Epoch : 0 | Batch : 450 | Train Loss : 0.007 | Train Acc : 1.000 | Train Recall : 1.000 
Epoch : 0 | Batch : 500 | Train Loss : 0.006 | Train Acc : 1.000 | Train Recall : 1.000 
Epoch : 0 | Batch : 550 

In [ ]:
# visualization loss
plt.plot(epoch_list, val_loss_list)
plt.xlabel("# of epochs")
plt.ylabel("Loss")
plt.title("CNN4: Loss vs # epochs")
plt.show()


# visualization accuracy
plt.plot(epoch_list, accuracy_list, color="red")
plt.xlabel("# of epochs")
plt.ylabel("Accuracy")
plt.title("CNN4: Accuracy vs # epochs")
plt.show()